uvicorn main:app --reload

ETL

In [16]:
import pandas as pd
import json
import patoolib
import ast

In [19]:
patoolib.extract_archive('steamGames.rar')
patoolib.extract_archive('steamGamesDev.rar')
patoolib.extract_archive('steamGamesGenres.rar')
patoolib.extract_archive('userItemCount.rar')
patoolib.extract_archive('userItems.rar')
patoolib.extract_archive('userReviews.rar')
patoolib.extract_archive('userReviewsExploded.rar')
patoolib.extract_archive('userItemsExploded.rar')
patoolib.extract_archive('steamGamesPrice.rar')

INFO patool: Extracting steamGames.rar ...
INFO patool: running "C:\Program Files\WinRAR\rar.EXE" x -- c:\Users\Admin\Code\MLOps-Steam\steamGames.rar
INFO patool:     with cwd=.\Unpack_vkfdj_a8, creationflags=134217728, input=
INFO patool: ... steamGames.rar extracted to `steamGames.csv'.


'steamGames.csv'

userItemsData=pd.read_csv('userItems.csv')
userItemCountData=pd.read_csv('userItemCount.csv')
userItemsExplodedData=pd.read_csv('userItemsExploded.csv')
steamGamesData=pd.read_csv('steamGames.csv',low_memory=False)
steamGamesPriceData=pd.read_csv('steamGamesPrice.csv')
userReviewsExplodedData=pd.read_csv('userReviewsExploded.csv')
userItemsDf=pd.DataFrame(userItemsData)
userItemCountDf=pd.DataFrame(userItemCountData)
userItemsExplodedDf=pd.DataFrame(userItemsExplodedData)
steamGamesDf=pd.DataFrame(steamGamesData)
steamGamesPriceDf=pd.DataFrame(steamGamesPriceData)
userReviewsExplodedDf=pd.DataFrame(userReviewsExplodedData)

Steam Games

In [4]:
with open('output_steam_games.json', 'r') as file:
  lines = file.readlines()

steamGamesData = [json.loads(line) for line in lines]

steamGames = pd.json_normalize(steamGamesData)

steamGamesDf=pd.DataFrame(steamGames)

In [5]:
steamGamesDf['release_date']=pd.to_datetime(steamGamesDf['release_date'],format='mixed',errors='coerce')
steamGamesDf['release_year'] = steamGamesDf['release_date'].dt.year.fillna(0).astype(int)

In [6]:
steamGamesDf.rename(columns={'id': 'item_id'}, inplace=True)

In [7]:
steamGamesDf.to_csv('steamGames.csv',index=False)

In [8]:
steamGamesPriceDf=steamGamesDf[['item_id','price']]
steamGamesPriceDf.to_csv('steamGamesPrice.csv',index=False)

In [9]:
steamGamesGenresDf=steamGamesDf[['item_id','genres','release_year']]
steamGamesGenresDf=steamGamesGenresDf.dropna(subset='genres')
steamGamesGenresDf.to_csv('steamGamesGenres.csv',index=False)

In [18]:
steamGamesDevDf=steamGamesDf[['item_id','developer','release_year']]
steamGamesDevDf=steamGamesDevDf.dropna(subset='developer')

In [19]:
steamGamesDevDf.to_csv('steamGamesDev.csv',index=False)

User Items

In [6]:
#Ruta del archivo JSON
data_list = []
file_path = 'australian_users_items.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
userItemsDf = pd.DataFrame(data_list)

In [7]:
userItemCount={'user_id': [], 'items_count': []}
userItemCountDf=pd.DataFrame(userItemCount)
userItemCountDf['user_id']=userItemsDf['user_id']
userItemCountDf['items_count']=userItemsDf['items_count']

In [8]:
del userItemsDf['items_count']
del userItemsDf['user_url']

In [9]:
userItemsExplodedDf=userItemsDf.explode('items').reset_index(drop=True)

In [10]:
userItemsExplodedDf['item_id']=userItemsExplodedDf['items'].apply(lambda x: x.get('item_id') if isinstance(x, dict) and 'item_id' in x else None)
userItemsExplodedDf['playtime']=userItemsExplodedDf['items'].apply(lambda x: x.get('playtime_forever') if isinstance(x, dict) and 'playtime_forever' in x else None)

In [12]:
del userItemsExplodedDf['steam_id']
del userItemsExplodedDf['items']

In [41]:
userItemsExplodedDf.to_csv('userItemsExploded.csv',index=False)
userItemsDf.to_csv('userItems.csv',index=False)
userItemCountDf.to_csv('userItemCount.csv',index=False)

In [14]:
userItemsDf

,user_id,steam_id,items
0,76561197970982479,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,76561197963445855,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,76561198002099482,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...
88305,76561198323066619,76561198323066619,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,76561198326700687,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,76561198328759259,[]
88308,76561198329548331,76561198329548331,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [16]:
userItemCountDf

,user_id,items_count
0,76561197970982479,277
1,js41637,888
2,evcentric,137
3,Riot-Punch,328
4,doctr,541
...,...,...
88305,76561198323066619,22
88306,76561198326700687,177
88307,XxLaughingJackClown77xX,0
88308,76561198329548331,7


In [36]:
steamGamesGenresDf

,item_id,genres
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
120440,773640,"[Casual, Indie, Simulation, Strategy]"
120441,733530,"[Casual, Indie, Strategy]"
120442,610660,"[Indie, Racing, Simulation]"
120443,658870,"[Casual, Indie]"


Def Developer

In [18]:
def developer(dev):
  steamGamesData=pd.read_csv('steamGames.csv',low_memory=False)
  steamGamesDf=pd.DataFrame(steamGamesData)
  filteredDf=steamGamesDf[steamGamesDf['developer']==dev]
  totalApps=filteredDf.groupby('release_year').size().reset_index(name='Cantidad de Items')
  filteredDf=filteredDf.dropna(subset=['price'])
  freeApps=filteredDf[filteredDf['price'].str.lower().str.contains('free')].groupby('release_year').size().reset_index(name='free_items')
  result = pd.merge(totalApps, freeApps, on='release_year', how='left').fillna(0)
  result.rename(columns={'release_year': 'Año'}, inplace=True)
  result['Contenido Free'] = ((result['free_items'] / result['Cantidad de Items']) * 100).round(2).astype(str) + '%'
  result=result[['Año','Cantidad de Items','Contenido Free']].reset_index(drop=True)
  return result

In [19]:
developer('AMPLITUDE Studios')

,Año,Cantidad de Items,Contenido Free
0,2012,1,0.0%
1,2014,3,0.0%
2,2015,9,22.22%
3,2016,2,0.0%
4,2017,8,87.5%


Def Userdata

In [30]:
def userData(userId):
  #Cargar CSV
  userRecommendData=pd.read_csv('userReviewsExploded.csv')
  userItemsData=pd.read_csv('userItemsExploded.csv')
  userItemCountData=pd.read_csv('userItemCount.csv')
  gamePriceData=pd.read_csv('steamGamesPrice.csv')
  #Convertir a DataFrame
  userRecommendDf=pd.DataFrame(userRecommendData)
  userItemsDf=pd.DataFrame(userItemsData)
  userItemCountDf=pd.DataFrame(userItemCountData)
  gamePriceDf=pd.DataFrame(gamePriceData)
  #Filtrar por usuario
  userRecommendDf=userRecommendDf[userRecommendDf['user_id']==userId].reset_index(drop=True)
  userItemsDf=userItemsDf[userItemsDf['user_id']==userId].reset_index(drop=True)
  userItemsDf=pd.merge(userItemsDf,gamePriceDf,how='left',on='item_id')
  userItemCountDf=userItemCountDf[userItemCountDf['user_id']==userId].reset_index(drop=True)
  #Componer DataFrame de respuesta
  resultData={'Usuario': [userId]}
  resultDf=pd.DataFrame(resultData)
  resultDf['Dinero Gastado']=pd.to_numeric(userItemsDf['price'], errors='coerce').sum()
  resultDf['% de recomendación']=((userRecommendDf['recommend'].mean()) * 100).round(2).astype(str) + '%'
  resultDf['Cantidad de Items']=userItemCountDf['items_count']
  return resultDf
userData('76561198043472122')

,Usuario,Dinero Gastado,% de recomendación,Cantidad de Items
0,76561198043472122,534.62,0.0%,88


Def UserForGenre

In [114]:
def userForGenre(genre):
  genre=genre.lower()
  gameGenreData=pd.read_csv('steamGamesGenres.csv')
  userItemsExplodedData=pd.read_csv('userItemsExploded.csv')
  gameGenreDf=pd.DataFrame(gameGenreData)
  userItemsExplodedDf=pd.DataFrame(userItemsExplodedData)
  gameGenreDf['genres']=gameGenreDf['genres'].str.lower()
  gameGenreDf=gameGenreDf[gameGenreDf['genres'].apply(lambda x: genre in x)]
  gameGenreDf=pd.merge(gameGenreDf[['item_id','release_year']],userItemsExplodedDf,how='left',on='item_id')
  gameGenreDf=gameGenreDf.dropna(subset='playtime')
  userId=gameGenreDf.groupby('user_id')['playtime'].sum().reset_index().sort_values(by='playtime',ascending=False).iloc[0,0]
  gameGenreDf=gameGenreDf[gameGenreDf['user_id']==userId].groupby('release_year')['playtime'].sum().reset_index().sort_values(by='release_year',ascending=False)
  gameGenreDf['playtime']=(gameGenreDf['playtime']/60).round(2)
  gameGenreDf.rename(columns={'release_year': 'Año'}, inplace=True)
  gameGenreDf.rename(columns={'playtime': 'Horas jugadas'}, inplace=True)
  resultPlaytime=gameGenreDf.to_dict(orient='records')
  result={'Usuario con más horas jugadas para Género ' + genre:userId,'Horas jugadas':resultPlaytime}
  return result
userForGenre('action')

{'Usuario con más horas jugadas para Género action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 2017, 'Horas Jugadas': 722.12},
  {'Año': 2016, 'Horas Jugadas': 492.93},
  {'Año': 2015, 'Horas Jugadas': 5208.52},
  {'Año': 2014, 'Horas Jugadas': 2178.18},
  {'Año': 2013, 'Horas Jugadas': 2007.68},
  {'Año': 2012, 'Horas Jugadas': 6304.93},
  {'Año': 2011, 'Horas Jugadas': 2581.6},
  {'Año': 2010, 'Horas Jugadas': 1301.38},
  {'Año': 2009, 'Horas Jugadas': 1805.43},
  {'Año': 2008, 'Horas Jugadas': 3.73},
  {'Año': 2007, 'Horas Jugadas': 1879.73},
  {'Año': 2006, 'Horas Jugadas': 14.93},
  {'Año': 2005, 'Horas Jugadas': 355.65},
  {'Año': 2004, 'Horas Jugadas': 2123.52},
  {'Año': 2003, 'Horas Jugadas': 127.88},
  {'Año': 2002, 'Horas Jugadas': 3.97},
  {'Año': 2001, 'Horas Jugadas': 0.22},
  {'Año': 2000, 'Horas Jugadas': 1177.4},
  {'Año': 1999, 'Horas Jugadas': 0.73},
  {'Año': 1998, 'Horas Jugadas': 0.0},
  {'Año': 1996, 'Horas Jugadas': 0.0},
  {'Año': 1995, 'Horas Jugadas': 3.62},
  {'

Def BestDeveloperYear

In [105]:
def bestDeveloperYear(year):
  userReviewsData=pd.read_csv('userReviewsExploded.csv')
  steamGamesDevData=pd.read_csv('steamGamesDev.csv')
  userReviewsDf=pd.DataFrame(userReviewsData)
  steamGamesDevDf=pd.DataFrame(steamGamesDevData)
  userReviewsDf = userReviewsDf[(userReviewsDf['recommend'] == True) & 
                               (userReviewsDf['sentiment_analysis'] == 2) & 
                               (userReviewsDf['review_year'] == year)]
  userReviewsDf['review_year']=userReviewsDf['review_year'].astype(int)
  userReviewsDf=userReviewsDf.groupby('item_id')['recommend'].count().reset_index()
  steamGamesDevDf=pd.merge(steamGamesDevDf[['item_id','developer']],userReviewsDf,how='left',on='item_id').sort_values(by='recommend',ascending=False)
  steamGamesDevDf=steamGamesDevDf.groupby('developer')['recommend'].sum().reset_index().sort_values(by='recommend',ascending=False)
  if steamGamesDevDf.iloc[1,1]!=0:
    first=steamGamesDevDf.iloc[0,0]
    second=steamGamesDevDf.iloc[1,0]
    third=steamGamesDevDf.iloc[2,0]
    result=[{'Puesto 1':first},{'Puesto 2':second},{'Puesto 3':third}]
  else:
    result=[{'Puesto 1':None},{'Puesto 2':None},{'Puesto 3':None}]
  return result
bestDeveloperYear(2010)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Tripwire Interactive'},
 {'Puesto 3': 'Facepunch Studios'}]

Def DeveloperReviewAnalysis

In [145]:
def developerReviewsAnalysis(dev):
  dev=dev.lower()
  userReviewsData=pd.read_csv('userReviewsExploded.csv')
  steamGamesDevData=pd.read_csv('steamGamesDev.csv')
  userReviewsDf=pd.DataFrame(userReviewsData)
  steamGamesDevDf=pd.DataFrame(steamGamesDevData)
  steamGamesDevDf=steamGamesDevDf[steamGamesDevDf['developer'].str.lower()== dev]
  userReviewsDf=userReviewsDf.groupby('item_id')['sentiment_analysis'].agg([('positive', lambda x: (x == 2).sum()),
    ('negative', lambda x: (x == 0).sum())]).reset_index()
  steamGamesDevDf=pd.merge(steamGamesDevDf,userReviewsDf,how='left',on='item_id')
  steamGamesDevDf=steamGamesDevDf.groupby('developer')[['positive','negative']].sum().reset_index()
  positive=steamGamesDevDf.iloc[0,1].astype(int).astype(str)
  negative=steamGamesDevDf.iloc[0,2].astype(int).astype(str)
  result={steamGamesDevDf.iloc[0,0]:['Negative = ' + negative,'Positive = ' + positive]}
  return result
developerReviewsAnalysis('Tripwire Interactive')

{'Tripwire Interactive': ['Negative = 109', 'Positive = 319']}